In [1]:
import pandas as pd
import numpy as np
import json
import os

filepath = os.path.join("../JSON/Historic_Request_ATL_20171222_14.json")
with open(filepath) as jsonfile:
    atl20180122_json = json.load(jsonfile)
    
    

FileNotFoundError: [Errno 2] No such file or directory: '../JSON/Historic_Request_ATL_20171222_14.json'

In [ ]:
#isolate data items
# Date Range A: Normal Operating Period 22nd Dec 2017 - 25th Jan 2018
# Date Range B: Shutdown Operating Period 22nd Dec 2018 - 25th Jan 2019
# Time Block: 2pm - 8pm
# Carrier: Delta Airlines (DL)
# Airports: MCO, ATL

# Data Fields: airport code, date, carrier code, 
# Filter by departureGateDelayMinutes, actualGateDeparture

# Need catch issue with departure date roll over where departure start time is less than departure end time
# Need to identify duplicate flight across adjacent days
# Example is Historic_Request_ATL_20190122_14.json flight 268

data = atl20180122_json["request"]
flightstatuses = atl20180122_json["flightStatuses"]

ATL_20180122_flights_df = pd.DataFrame(index=np.arange(0, 0), columns=('Counter', 'ID', 'Carrier', 'Number', 'Airport', 'Date', 'Status', 'Scheduled Hour', 'Scheduled Mins', 'Actual Hour', 'Actual Mins', 'Delay') )

Flight_Counter = 0
Flights = []

for getFlight in atl20180122_json['flightStatuses']:
    Flight_Counter = Flight_Counter + 1
    print(f"Flight Counter: {Flight_Counter}")
    myFlightID = getFlight['flightId']
    print(f"FlightID: {myFlightID}")
    myFlightCarrier = getFlight['carrierFsCode']
    print(f"Carrier: {myFlightCarrier}")
    myFlightNum = getFlight['flightNumber']
    print(f"Flight#: {myFlightNum}")
    myFlightDepAirport = getFlight['departureAirportFsCode']
    print(f"Airport: {myFlightDepAirport}")
    myFlightDepDateTime = getFlight['departureDate']['dateLocal']
    myFlightDepDate = myFlightDepDateTime.split("T")[0]
    print(f"Date: {myFlightDepDate}")
    myFlightStatus = getFlight['status']
    print(f"Status: {myFlightStatus}")
    
    try:
        myFlightScheduledDep = getFlight['operationalTimes']['scheduledGateDeparture']['dateLocal']
        myFlightScheduledDepTime = myFlightScheduledDep.split("T")[1]
        myFlightShedDepHour = myFlightScheduledDepTime.split(":")[0]
        myFlightShedDepMinutes = myFlightScheduledDepTime.split(":")[1]
        print(f"Scheduled Dep Time: {myFlightScheduledDepTime} Hour: {myFlightShedDepHour} Minutes: {myFlightShedDepMinutes}")
    
        myFlightActualDep = getFlight['operationalTimes']['actualGateDeparture']['dateLocal']
        myFlightActualDepTime = myFlightActualDep.split("T")[1]
        myFlightActualDepHour = myFlightActualDepTime.split(":")[0]
        myFlightActualDepMinutes = myFlightActualDepTime.split(":")[1]
        print(f"Actual Dep Time: {myFlightActualDepTime} Hour: {myFlightActualDepHour} Minutes: {myFlightActualDepMinutes}")
    
        myFlightDepStartTime = (int(myFlightShedDepHour) * 60) + int(myFlightShedDepMinutes)
        myFlightDepEndTime = (int(myFlightActualDepHour) * 60) + int(myFlightActualDepMinutes)
        myFlightDelayMinutes = myFlightDepEndTime - myFlightDepStartTime
        #print(f"Start Time: {myFlightDepStartTime}")
        #print(f"End Time: {myFlightDepEndTime}")
        print(f"Delay (Minutes): {myFlightDelayMinutes}")
    except:
        print(f"No Gate Depature Delay Data")
        myFlightDelayMinutes = 0
        print(f"Delay (Minutes): {myFlightDelayMinutes}")
        
    print("\n")
    ATL_20180122_flights_df.loc[Flight_Counter] = [Flight_Counter, myFlightID, myFlightCarrier, myFlightNum, myFlightDepAirport, myFlightDepDate, myFlightStatus, myFlightShedDepHour, myFlightShedDepMinutes, myFlightActualDepHour, myFlightActualDepMinutes, myFlightDelayMinutes]


In [ ]:
# Describe() for Date Range A
print(ATL_20180122_flights_df)


In [ ]:
# Describe() for Date Range B
# Compare total number of flights, average flights per day

# Plot average delay by hour in a random day during Date Range A
# Plot average delay by hour in a random day during Date Range B